ACTUALIZA LAS PROPIEDADES QUE TIENEN MIDIFICACIONES

In [1]:
import ee
ee.Authenticate()

True

In [2]:
import ee
import geemap
# metodo de clasificacion
import jenkspy

import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Polygon

import pandas as pd
import numpy as np

import openpyxl

In [3]:
ee.Initialize()

In [4]:
ruta_catastro='projects/ee-bismarksr17/assets/catastro_2021'
ruta_ndvi = 'projects/ee-bismarksr17/assets/NDVI-2021-03'

CODIGO_PROPIEDAD='COD_PROP'
NOM_PROPIEDAD='PROPIEDAD'
CODIGO_CANHERO='CODCA_1'
NOM_CANHERO='CAÑERO'

VARIEDAD='Variedad'
ESTADO='RENOVACION'
SUPERFICIE='SUPERFICIE'

In [5]:
def asig_cat(label):
    if label == 1:
        return 0
    else:
        return label * 10

vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# parametro de visualizacion LOTES RENOVACION
vis_params_renov = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

## Eliminar propiedades sobra

In [26]:
# props que presentan irregularidades
lista_eliminar = [106, 622, 1080, 1172, 1302, 1413, 1480, 1540, 1626, 1986,
    1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
    1998, 1999, 2001, 2002, 1421, 1962, 1289, 988, 986, 1446,
    1454, 637, 15, 636, 92, 1735, 880, 328, 19, 201, 114, 437,
    216, 1447, 365, 47, 123, 249, 132, 17, 1491, 1015, 85, 325,
    367, 491, 478, 1081, 744, 752, 313, 616, 1265, 1482, 1743]

In [7]:
df_intersects = gpd.read_file('INTERSECT_2.shp')

In [28]:
df_data = pd.read_excel('DATA_EST.xlsx')

### Eliminar props de DATA_EST.xlsx

In [29]:
df_data = df_data[~df_data['cod_prop'].isin(lista_eliminar)]

In [30]:
df_data.to_excel('data_new.xlsx')

### Eliminar props de INTERSECTS.shp

In [14]:
lista_eliminar = [str(i) for i in lista_eliminar]

In [15]:
df_intersects = df_intersects[~df_intersects['unidad_01'].isin(lista_eliminar)]

In [6]:
ruta_fases = 'props_2021.xlsx'

In [7]:
df = pd.read_excel(ruta_fases)

In [9]:
df.head(5)

,COD_PROP
0,1
1,4
2,5
3,6
4,8


In [11]:
lista_props = list(df['COD_PROP'])

In [12]:
len(lista_props)

1268

In [45]:
print(lista_props)

[1, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50, 53, 54, 55, 57, 61, 62, 63, 66, 67, 68, 71, 72, 74, 76, 78, 79, 80, 83, 84, 85, 92, 93, 94, 98, 100, 104, 106, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 122, 123, 125, 126, 127, 128, 129, 132, 133, 135, 137, 139, 140, 141, 142, 145, 146, 147, 149, 150, 155, 156, 159, 161, 162, 164, 168, 171, 172, 173, 174, 176, 178, 179, 182, 183, 184, 185, 186, 187, 188, 189, 195, 197, 198, 201, 205, 206, 209, 211, 212, 213, 216, 217, 218, 219, 221, 222, 225, 226, 227, 228, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 246, 247, 249, 250, 251, 253, 256, 257, 258, 259, 263, 264, 265, 267, 270, 271, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 313, 314, 318, 320, 324, 325, 3

In [50]:
lista_props = [236, 237, 238, 239, 240, 241, 242, 244, 246, 247, 249, 250, 251, 253, 256, 257, 258, 259, 263, 264, 265, 267, 270, 271, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 313, 314, 318, 320, 324, 325, 326, 327, 328, 329, 330, 332, 333, 341, 342, 343, 344, 345, 348, 349, 350, 355, 358, 360, 361, 365, 367, 368, 369, 371, 374, 375, 376, 377, 378, 384, 386, 389, 391, 392, 395, 396, 398, 399, 400, 403, 404, 405, 406, 407, 408, 411, 414, 415, 416, 417, 418, 419, 421, 423, 424, 426, 429, 435, 437, 439, 440, 441, 442, 444, 445, 447, 449, 450, 454, 458, 459, 466, 468, 469, 471, 472, 473, 476, 478, 483, 484, 485, 486, 487, 488, 489, 490, 491, 493, 494, 495, 496, 499, 500, 501, 502, 503, 504, 505, 507, 509, 510, 511, 512, 513, 514, 515, 516, 517, 519, 520, 521, 522, 523, 524, 525, 526, 527, 529, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 544, 545, 546, 547, 548, 549, 550, 551, 552, 554, 555, 556, 558, 559, 560, 566, 567, 570, 571, 572, 573, 574, 575, 576, 577, 579, 582, 583, 585, 586, 588, 589, 590, 591, 592, 593, 596, 597, 600, 601, 602, 603, 605, 606, 607, 608, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 627, 628, 629, 630, 631, 632, 633, 634, 636, 637, 638, 639, 640, 641, 642, 643, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 658, 659, 660, 661, 662, 663, 664, 665, 667, 668, 669, 671, 673, 674, 675, 676, 677, 678, 679, 681, 682, 683, 685, 687, 690, 691, 692, 693, 695, 696, 697, 698, 699, 700, 702, 703, 704, 705, 706, 707, 708, 709, 710, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 732, 733, 734, 737, 739, 741, 742, 744, 745, 746, 747, 749, 750, 751, 752, 753, 755, 756, 757, 758, 759, 761, 763, 765, 766, 767, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 787, 788, 789, 790, 791, 792, 794, 796, 797, 798, 801, 802, 803, 804, 805, 806, 807, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 820, 823, 824, 825, 827, 828, 829, 830, 831, 832, 833, 834, 836, 838, 839, 840, 842, 843, 844, 845, 846, 848, 849, 851, 852, 853, 855, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 877, 878, 879, 880, 882, 883, 886, 887, 888, 889, 891, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 908, 909, 910, 912, 913, 914, 915, 916, 918, 919, 921, 922, 923, 924, 925, 926, 927, 928, 930, 934, 936, 937, 938, 939, 941, 942, 943, 944, 945, 947, 949, 950, 951, 952, 953, 954, 956, 957, 958, 961, 962, 963, 964, 966, 968, 969, 970, 971, 972, 973, 974, 975, 976, 978, 979, 980, 981, 982, 984, 986, 988, 989, 990, 991, 992, 993, 994, 995, 996, 999, 1000, 1001, 1002, 1005, 1006, 1009, 1010, 1011, 1013, 1014, 1015, 1016, 1018, 1020, 1021, 1022, 1024, 1025, 1026, 1028, 1033, 1034, 1035, 1036, 1038, 1039, 1040, 1041, 1042, 1043, 1045, 1046, 1047, 1049, 1050, 1051, 1052, 1053, 1054, 1057, 1064, 1065, 1067, 1069, 1070, 1073, 1074, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1099, 1100, 1102, 1104, 1106, 1107, 1108, 1109, 1111, 1113, 1114, 1115, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1128, 1130, 1131, 1136, 1139, 1140, 1142, 1144, 1145, 1146, 1147, 1148, 1149, 1151, 1153, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1165, 1166, 1168, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1189, 1190, 1193, 1195, 1198, 1199, 1200, 1201, 1203, 1204, 1205, 1206, 1207, 1208, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1219, 1220, 1221, 1222, 1225, 1227, 1228, 1229, 1231, 1233, 1235, 1238, 1240, 1241, 1242, 1243, 1244, 1248, 1249, 1250, 1251, 1252, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1263, 1264, 1265, 1266, 1268, 1269, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1289, 1290, 1291, 1292, 1294, 1295, 1297, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1313, 1314, 1315, 1322, 1323, 1325, 1326, 1327, 1330, 1331, 1332, 1333, 1335, 1337, 1338, 1339, 1340, 1342, 1343, 1344, 1345, 1346, 1347, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1374, 1375, 1376, 1377, 1378, 1380, 1381, 1382, 1384, 1385, 1386, 1389, 1392, 1393, 1394, 1397, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1416, 1421, 1422, 1423, 1424, 1425, 1426, 1428, 1431, 1432, 1433, 1434, 1436, 1438, 1440, 1441, 1442, 1443, 1446, 1447, 1450, 1451, 1452, 1453, 1454, 1456, 1457, 1458, 1459, 1460, 1461, 1464, 1466, 1467, 1469, 1470, 1472, 1474, 1475, 1476, 1478, 1480, 1481, 1482, 1483, 1484, 1488, 1489, 1491, 1492, 1494, 1495, 1498, 1500, 1502, 1503, 1505, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1516, 1518, 1520, 1522, 1523, 1524, 1526, 1528, 1531, 1535, 1536, 1537, 1539, 1540, 1541, 1542, 1544, 1545, 1547, 1549, 1550, 1551, 1553, 1554, 1556, 1558, 1559, 1561, 1562, 1564, 1566, 1567, 1568, 1570, 1572, 1573, 1574, 1576, 1577, 1578, 1579, 1580, 1582, 1583, 1585, 1586, 1587, 1588, 1591, 1592, 1593, 1594, 1596, 1597, 1598, 1600, 1602, 1604, 1605, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1627, 1628, 1629, 1630, 1631, 1633, 1635, 1637, 1638, 1639, 1640, 1642, 1643, 1644, 1645, 1646, 1648, 1649, 1650, 1652, 1653, 1654, 1655, 1656, 1658, 1659, 1660, 1661, 1662, 1663, 1665, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1682, 1683, 1684, 1685, 1686, 1688, 1689, 1690, 1693, 1694, 1695, 1696, 1699, 1700, 1701, 1702, 1703, 1704, 1705, 1706, 1707, 1708, 1709, 1711, 1712, 1716, 1718, 1719, 1720, 1721, 1722, 1723, 1724, 1726, 1730, 1731, 1732, 1733, 1735, 1736, 1737, 1738, 1739, 1740, 1742, 1743, 1744, 1745, 1748, 1750, 1751, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1767, 1768, 1770, 1771, 1772, 1774, 1775, 1776, 1777, 1778, 1780, 1781, 1782, 1783, 1784, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1797, 1798, 1799, 1800, 1801, 1802, 1805, 1806, 1807, 1808, 1811, 1812, 1815, 1816, 1817, 1818, 1819, 1820, 1823, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1844, 1845, 1848, 1849, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890, 1891, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1944, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1960, 1961, 1962, 1963, 1964, 1968, 1969, 1970, 1971, 1972, 1973, 1975, 1976, 1977, 1978, 1979, 1980, 1982, 1984, 1985]
#lista_props = [2075]

In [51]:
len(lista_props)

1261

In [14]:
df_intersects = gpd.read_file('INTERSECT_3.shp')

In [52]:
contador = 0
for cod_prop in lista_props:
    print('inicio:', cod_prop)
    propiedad = ee.FeatureCollection(ruta_catastro)\
                .filter(ee.Filter.eq(CODIGO_PROPIEDAD, cod_prop))
    lotes_canha = propiedad.filter(ee.Filter.neq(VARIEDAD, ESTADO))
    lotes_renov = propiedad.filter(ee.Filter.eq(VARIEDAD, ESTADO))
    NDVI = ee.Image(ruta_ndvi)
    NDVI_clip = NDVI.clip(lotes_canha.geometry())
    # crea un sample de los valore de pixel del NDVI
    NDVI_values = NDVI_clip.sampleRegions(lotes_canha.geometry())
    # reduce el resultado a valores de NDVI
    pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['b1']).get('list').getInfo()
    # aplica metodo Jenks
    breaks = jenkspy.jenks_breaks(pixel_values, n_classes=8)
    NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)
    NDVI_class_clip = NDVI_class.clip(lotes_canha.geometry())
    clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)
    vector = clasify.reduceToVectors(**{
        'geometry': lotes_canha.geometry(),
        'crs': clasify.projection(),
        'scale': 10,
        'geometryType': 'polygon',
        'eightConnected': False
    })
    
    DF_VECTOR = vector
    
    lotes_local = geemap.ee_to_gdf(lotes_canha)
    
    lista = vector.toList(vector.size()).getInfo()
    lista_vector=[]
    for item in lista:
        dic = {'geometry':Polygon(item['geometry']['coordinates'][0]), 'count':item['properties']['count'], 'label':item['properties']['label']}
        lista_vector.append(dic)
    
    vector_local = gpd.GeoDataFrame(lista_vector)
    
    intersect = overlay(lotes_local, vector_local, how="intersection")
    intersect.crs = "EPSG:4326"
    intersect = intersect.to_crs(epsg=32720)
    intersect['area_2'] = intersect['geometry'].area/10000
    
    area_01 = intersect['area_2'].sum()
    area_02 = lotes_local[SUPERFICIE].sum()
    area_diff = (area_02 - area_01)/len(intersect)
    intersect['area_2'] = intersect['area_2'] + area_diff
    
    DF = intersect.copy()
    dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)
    dina2 = dina.reset_index()
    
    dina2['tch'] = dina2['label'].apply(asig_cat)
    
    # Cargar el archivo de Excel existente
    wb = openpyxl.load_workbook('DATA_EST.xlsx')
    # Seleccionar la hoja de trabajo
    ws = wb['data']
    
    for i in range(0, len(dina2)):
        new_row = [dina2.iloc[i,0], dina2.iloc[i,1], dina2.iloc[i,2], dina2.iloc[i,3], dina2.iloc[i,4], dina2.iloc[i,5], dina2.iloc[i,6]]
        ws.append(new_row)
    wb.save('DATA_EST.xlsx')
    
    df_intersects = pd.concat([df_intersects, DF], ignore_index=True)
    
    contador = contador + 1 
    print('fin: ', cod_prop)
    print('___________________CONTADOR: ', contador)
    
print('FIN......!')

inicio: 236


C:\Users\Usuario\AppData\Local\Temp\ipykernel_17136\3576922828.py:46: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersect = overlay(lotes_local, vector_local, how="intersection")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17136\3576922828.py:57: FutureWarning: The provided callable <function sum at 0x0000025F54DA6560> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)


fin:  236
___________________CONTADOR:  1
inicio: 237


C:\Users\Usuario\AppData\Local\Temp\ipykernel_17136\3576922828.py:46: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersect = overlay(lotes_local, vector_local, how="intersection")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17136\3576922828.py:57: FutureWarning: The provided callable <function sum at 0x0000025F54DA6560> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)


fin:  237
___________________CONTADOR:  2
inicio: 238


...

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17136\3576922828.py:46: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersect = overlay(lotes_local, vector_local, how="intersection")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17136\3576922828.py:57: FutureWarning: The provided callable <function sum at 0x0000025F54DA6560> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)


fin:  343
___________________CONTADOR:  79
inicio: 344


C:\Users\Usuario\AppData\Local\Temp\ipykernel_17136\3576922828.py:46: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersect = overlay(lotes_local, vector_local, how="intersection")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17136\3576922828.py:57: FutureWarning: The provided callable <function sum at 0x0000025F54DA6560> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)


fin:  344
___________________CONTADOR:  80
inicio: 345


EEException: Image.clip: The geometry for image clipping must not be empty.

In [58]:
len(df_intersects)

757682

In [60]:
df_intersects.to_file("INTERSECT_2022.shp", driver="ESRI Shapefile")

In [59]:
df_intersects['area_2'].sum()

50300.66341328999